# Guide to Using the LastMile Instrumentor for LlamaIndex
This guide will walk you through the process of using the LastMile Instrumentor for LlamaIndex. The LastMile Instrumentor is a powerful tool that allows you to trace and debug your LlamaIndex applications, providing valuable insights into their performance and behavior.



Prerequisites
Before we get started, make sure you have the following prerequisites installed:

Python 3.x
LlamaIndex
OpenAI API key

To install the required dependencies, run the first  cell

In [13]:
!pip install llama-index-embeddings-openai
!pip install llama-index-embeddings-openai --upgrade

%pip install -q html2text llama-index pandas pyarrow tqdm
%pip install -q llama-index-readers-web
%pip install -q llama-index-callbacks-openinference
!pip install openai --upgrade
!pip install "tracing-auto-instrumentation[llama-index]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Step 1: Import Required Libraries

First, let's import the necessary libraries for our project.

In [6]:
import os
from getpass import getpass

import dotenv
import llama_index.core

from tracing_auto_instrumentation import LlamaIndexCallbackHandler
import textwrap

## Step 2: Configure the LastMile Instrumentor

Next, we need to configure the LastMile Instrumentor by setting the global handler for LlamaIndex.

In [12]:
import llama_index.core

from tracing_auto_instrumentation import LlamaIndexCallbackHandler

llama_index.core.global_handler = LlamaIndexCallbackHandler(
    project_name="LlamaIndex Paul Graham QA",
)

2024-05-21 21:03:10,892 - Overriding of current TracerProvider is not allowed
2024-05-21 21:03:10,893 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-21 21:03:10,992 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=LlamaIndex+Paul+Graham+QA HTTP/1.1" 200 360


# Step 3: Set Up OpenAI API Key

In [8]:
dotenv.load_dotenv()
if os.getenv("OPENAI_API_KEY") is None:
    os.environ["OPENAI_API_KEY"] = getpass(
        "Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n"
    )
assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


# Step 4: Load and Process Documents


In [9]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader().load_data(
    [
        "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"
    ]
)

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)

2024-05-21 21:01:52,612 - Starting new HTTPS connection (1): raw.githubusercontent.com:443
2024-05-21 21:01:52,641 - https://raw.githubusercontent.com:443 "GET /run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt HTTP/1.1" 200 28419
2024-05-21 21:01:52,728 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:01:52,729 - > Adding chunk: What I Worked On

February 2021

Before college...
2024-05-21 21:01:52,730 - > Adding chunk: I couldn't have put this into words when I was ...
2024-05-21 21:01:52,730 - > Adding chunk: So I looked around to see what I could salvage ...
2024-05-21 21:01:52,730 - > Adding chunk: I remember when my friend Robert Morris got kic...
2024-05-21 21:01:52,731 - > Adding chunk: Except hardly anyone else painted her besides m...
2024-05-21 21:01:52,731 - > Adding chunk: Now they wanted a Lisp hacker to write things i...
2024-05-21 21:01:52,731 - > Adding chunk: For example, when you see a painting tha

# Step 5: Create an Index and Query Engine

In [10]:
from llama_index.embeddings.openai.base import OpenAIEmbedding

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()


2024-05-21 21:01:57,317 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:01:57,318 - > Adding chunk: What I Worked On

February 2021

Before college...
2024-05-21 21:01:57,319 - > Adding chunk: I couldn't have put this into words when I was ...
2024-05-21 21:01:57,319 - > Adding chunk: So I looked around to see what I could salvage ...
2024-05-21 21:01:57,319 - > Adding chunk: I remember when my friend Robert Morris got kic...
2024-05-21 21:01:57,320 - > Adding chunk: Except hardly anyone else painted her besides m...
2024-05-21 21:01:57,320 - > Adding chunk: Now they wanted a Lisp hacker to write things i...
2024-05-21 21:01:57,320 - > Adding chunk: For example, when you see a painting that looks...
2024-05-21 21:01:57,320 - > Adding chunk: It seemed like the web would do the same for th...
2024-05-21 21:01:57,321 - > Adding chunk: (Had I diligently set aside the proper proporti...
2024-05-21 21:01:57,321 - > Adding chunk: We had no idea what busines

# Step 6: Query the Index

In [11]:
max_characters_per_line = 80
queries = [
    "What did Paul Graham do growing up?",
    "When and how did Paul Graham's mother die?",
    "What, in Paul Graham's opinion, is the most distinctive thing about YC?",
    "When and how did Paul Graham meet Jessica Livingston?",
    "What is Bel, and when and where was it written?",
]
for query in queries:
    response = query_engine.query(query)
    print("Query")
    print("=====")
    print(textwrap.fill(query, max_characters_per_line))
    print()
    print("Response")
    print("========")
    print(textwrap.fill(str(response), max_characters_per_line))
    print()

2024-05-21 21:02:06,294 - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x29c8f15a0>, 'json_data': {'input': ['What did Paul Graham do growing up?'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
2024-05-21 21:02:06,295 - Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2024-05-21 21:02:06,296 - close.started
2024-05-21 21:02:06,296 - close.complete
2024-05-21 21:02:06,297 - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
2024-05-21 21:02:06,308 - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x2996360b0>
2024-05-21 21:02:06,308 - start_tls.started ssl_context=<ssl.SSLContext object at 0x2996b3040> server_hostname='api.openai.com' timeout=60.0
2024-05-21 21:02:06,318 - start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x29a6ce350>
2024-05-21 21:02:06

Query
=====
What did Paul Graham do growing up?

Response
Paul Graham grew up writing short stories and programming.



2024-05-21 21:02:08,223 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:08,253 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:08,286 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:08,289 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': 'Context information is below.\n---------------------\nI was invited to give a talk at a Lisp conference, so I gave one about how we\'d used Lisp at Viaweb. Afterward 

Query
=====
When and how did Paul Graham's mother die?

Response
Paul Graham's mother died when he was 18 years old.



2024-05-21 21:02:09,646 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:09,682 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:09,685 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': 'Context information is below.\n---------------------\nObviously software and venture capital will be, but who would have predicted that essay writing would be?\n\n[13] Y Combinator was not the original name. At first we were called Cambridge Seed. But we didn\

Query
=====
What, in Paul Graham's opinion, is the most distinctive thing about YC?

Response
The most distinctive thing about Y Combinator, according to Paul Graham, is that
instead of deciding for himself what to work on, the problems come to him. Every
6 months, a new batch of startups brings their problems, which then become the
problems of YC. This engagement with a variety of startup problems and the
effective founders make YC's approach unique in Graham's opinion.



2024-05-21 21:02:12,181 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:12,212 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:12,243 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:12,247 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}, {'role': 'user', 'content': 'Context information is below.\n---------------------\nI was invited to give a talk at a Lisp conference, so I gave one about how we\'d used Lisp at Viaweb. Afterward 

Query
=====
When and how did Paul Graham meet Jessica Livingston?

Response
Paul Graham met Jessica Livingston at a big party at his house in October 2003.



2024-05-21 21:02:13,595 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:13,599 - > Top 2 nodes:
> [Node 71b14800-e639-4afb-99fa-2d2f5ab2eead] [Similarity score:             0.831573] It took 4 years, from March 26, 2015 to October 12, 2019. It was fortunate that I had a precisely...
> [Node 13cbd219-f404-49a9-9e69-9d1efff9b20e] [Similarity score:             0.764116] (I still talk to alumni and to new startups working on things I'm interested in, but that only ta...
2024-05-21 21:02:13,625 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:13,675 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:13,705 - https://lastmileai.dev:443 "POST /api/trace/create HTTP/1.1" 200 10
2024-05-21 21:02:13,709 - Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': "You are an expert Q&A system that is trusted

Query
=====
What is Bel, and when and where was it written?

Response
Bel is a new Lisp that was written in Arc. It was written over a period of 4
years, from March 26, 2015 to October 12, 2019. Most of Bel was written in
England.

